In [1]:
from openai import AzureOpenAI

import os
import sys
import json
import re
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [12]:
v1 = pd.read_parquet("synthetic_data_github.parquet")
v2 = pd.read_parquet("synthetic_bertil.parquet")
v = pd.concat([v1, v2], ignore_index = True)

In [13]:
v

,description,summary
0,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,"*Sjukdomshistoria*\nHypertoni, hypotyreos, hyp..."
1,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,<Sjukdomshistoria>\n58-årig kvinnlig patient m...
2,sokorsak: Dyspné \nkon: K \nalder: 77 \nsocial...,*Patientens namn och personnummer är fingerade...
3,sokorsak: BrastSmarta \nkon: M \nalder: 51 \ns...,"*Sjukdomshistoria*\nHypertoni, Typ 2 diabetes...."
4,sokorsak: BukSmarta \nkon: K \nalder: 20 \nsoc...,*Sjukdomshistoria*\nMisstänkt appendicit (K379...
...,...,...
68,"Daganteckning 24-05-12 kl 0815, Läk Hanna Solb...","Sjukdomshistoria (Patientens diagnoser, sjukdo..."
69,Besöksanteckning Neurologimottagningen 23-07-1...,"Sjukdomshistoria (Patientens diagnoser, sjukdo..."
70,"Kommentar till undersökning, 23-08-15, Överläk...","Sjukdomshistoria (Patientens diagnoser, sjukdo..."
71,"Återbesök neurodagvård, strokeenhet 23-04-25, ...","Sjukdomshistoria (Patientens diagnoser, sjukdo..."


In [14]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [15]:
instructions = """You are a helpful assistant, that helps to create synthetic data. 
                  You will be provided with a medical note of a patient and a summary.
                  The description and the summary are both in Swedish, and you should
                  answer in Swedish.
                  Please generate another note-summary pair for a new, realistic fake patient.
                  Remember to use the same template as the provided example for the summary.
                  """

In [16]:
responses = []

In [18]:
for i,(x,y) in v.iterrows():
    
    now = datetime.now()
    one_hour_later = now + timedelta(hours=1)
    current_time = one_hour_later.strftime("%H:%M:%S")
    
    print(i,"-",current_time)
    
    question = f"""Please create a similar patient <Description>{x}</Description><Summary>{y}</Summary>""" + \
    """Respond with the same template, using tags."""
    
    messages = [{"role": "system", "content": instructions}, {"role": "user", "content": question}]
    
    response = client.chat.completions.create(
      model="gpt4",
      #response_format={ "type": "json_object" },
      messages=messages
    )
    
    match = re.split(r"<[^>]*>",response.choices[0].message.content)
    if len(match) == 5:
        desc = match[1]
        summ = match[3]
        responses.append((desc,summ))
    elif len(match) == 1:
        match = re.split(r"Sjukdomshistoria", txt,maxsplit=1)
        desc = match[0]
        summ = 'Sjukdomshistoria'+match[1]
        responses.append((desc,summ))
    
    
        
    

0 - 10:40:25
1 - 10:42:16
2 - 10:44:41
3 - 10:46:52
4 - 10:47:33
5 - 10:48:31
6 - 10:49:31
7 - 10:51:45
8 - 10:53:01
9 - 10:54:25
10 - 10:56:39
11 - 10:57:56
12 - 10:58:21
13 - 10:59:51
14 - 11:02:29
15 - 11:06:51
16 - 11:07:45
17 - 11:09:01
18 - 11:09:55
19 - 11:11:48
20 - 11:12:38
21 - 11:13:19
22 - 11:14:29
23 - 11:15:28
24 - 11:16:42
25 - 11:17:28
26 - 11:17:52
27 - 11:19:06
28 - 11:20:36
29 - 11:22:01
30 - 11:23:10
31 - 11:23:54
32 - 11:24:34
33 - 11:25:18
34 - 11:26:16
35 - 11:28:23
36 - 11:29:11
37 - 11:29:54
38 - 11:32:01
39 - 11:32:59
40 - 11:34:01
41 - 11:34:58
42 - 11:35:32
43 - 11:36:05
44 - 11:37:00
45 - 11:37:48
46 - 11:39:48
47 - 11:41:14
48 - 11:42:24
49 - 11:43:17
50 - 11:45:03
51 - 11:46:24
52 - 11:46:48
53 - 11:47:18
54 - 11:48:30
55 - 11:49:47
56 - 11:51:03
57 - 11:53:18
58 - 11:54:39
59 - 11:55:53
60 - 11:57:02
61 - 11:59:00
62 - 11:59:47
63 - 12:02:45
64 - 12:03:25
65 - 12:04:43
66 - 12:05:42
67 - 12:07:25
68 - 12:09:00
69 - 12:11:11
70 - 12:13:11
71 - 12:14:13
72

In [23]:
len(responses)

57

In [13]:
len(match)

1

In [9]:
response.choices[0].message.content

'```plaintext\nInskrivningsanteckning 24-04-10, Avd 34/12 Kardiologi kl 09.15, Läkare Sara Bergman:\nInskrivningsorsak: Akut myokardinfarkt, genomgången PCI\nTidigare/Nuvarande sjukdomar: Rökning (upphört för 5 år sedan), typ 2-diabetes, behandlad med Metformin. Ingen känd hjärtsjukdom i familjen. Ej känt med några allergier.\nSocialt: Pensionär, tidigare lärare. Gift, tre vuxna barn. Lever ett stillasittande liv.\nAktuella läkemedel: \nTabl Metformin 850 mg 2 x 1 \nTabl Trombyl 75 mg 1 x 1 (påbörjad efter PCI)\nAktuellt: Vaknade tidigt på morgonen med tryck över bröstet och andfåddhet. Efter att ha tvekat tog patienten kontakt med 1177 som rådde omedelbar kontakt med ambulans. Vid ankomst till sjukhuset direktransport till koronarintervention där PCI genomförs framgångsrikt.\nSTATUS:\nAllmäntillstånd: Vaken, RLS 1, trött men orienterad.\nHjärta: Lätt tackykard, men rytmisk och utan extraljud.\nLunga: Normalt andningsljud, ingen andnöd i vila.\nBedömning/åtgärd: 68-årig patient med tid

In [24]:
df = pd.DataFrame(responses,columns=["description","summary"])
df.to_parquet("synthetic_2nd.parquet",compression=None)

In [30]:
v_130 = pd.concat([v,df],ignore_index=True)

In [29]:
v_130

,description,summary
0,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,"*Sjukdomshistoria*\nHypertoni, hypotyreos, hyp..."
1,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,<Sjukdomshistoria>\n58-årig kvinnlig patient m...
2,sokorsak: Dyspné \nkon: K \nalder: 77 \nsocial...,*Patientens namn och personnummer är fingerade...
3,sokorsak: BrastSmarta \nkon: M \nalder: 51 \ns...,"*Sjukdomshistoria*\nHypertoni, Typ 2 diabetes...."
4,sokorsak: BukSmarta \nkon: K \nalder: 20 \nsoc...,*Sjukdomshistoria*\nMisstänkt appendicit (K379...
...,...,...
125,"Daganteckning 24-08-15 kl 0845, Läk Magnus Jon...","Sjukdomshistoria (Patientens diagnoser, sjukdo..."
126,Besöksanteckning Kardiologimottagningen 23-08-...,"Sjukdomshistoria (Patientens diagnoser, sjukdo..."
127,"Kommentar till undersökning, 23-11-20, Överläk...","Sjukdomshistoria (Patientens diagnoser, sjukdo..."
128,"Uppföljningsbesök diabetesmottagning, Sahlgren...","Sjukdomshistoria (Patientens diagnoser, sjukdo..."


In [ ]:
new_responses = []
for i,(x,y) in v_130.iterrows():
    
    now = datetime.now()
    one_hour_later = now + timedelta(hours=1)
    current_time = one_hour_later.strftime("%H:%M:%S")
    
    print(i,"-",current_time,"-", len(new_responses))
    
    question = f"""Please create a similar patient <Description>{x}</Description><Summary>{y}</Summary>""" + \
    """Respond with the same template, using tags."""
    
    messages = [{"role": "system", "content": instructions}, {"role": "user", "content": question}]
    
    response = client.chat.completions.create(
      model="gpt4",
      #response_format={ "type": "json_object" },
      messages=messages
    )
    
    match = re.split(r"<[^>]*>",response.choices[0].message.content)
    if len(match) == 5:
        desc = match[1]
        summ = match[3]
        new_responses.append((desc,summ))
    elif len(match) == 1:
        match = re.split(r"Sjukdomshistoria", txt,maxsplit=1)
        desc = match[0]
        summ = 'Sjukdomshistoria'+match[1]
        new_responses.append((desc,summ))
    
    
        
    

0 - 15:19:02 - 0
1 - 15:20:30 - 1
2 - 15:21:58 - 1
3 - 15:23:49 - 1
4 - 15:24:39 - 2
5 - 15:25:52 - 3
6 - 15:27:21 - 4
7 - 15:28:47 - 4
8 - 15:29:49 - 5
9 - 15:30:40 - 6
10 - 15:31:51 - 7
11 - 15:32:39 - 8
12 - 15:33:02 - 8
13 - 15:34:41 - 9
14 - 15:36:00 - 10
15 - 15:36:55 - 11
16 - 15:37:49 - 12
17 - 15:38:27 - 13
18 - 15:40:06 - 14
19 - 15:40:57 - 15
20 - 15:42:05 - 16
21 - 15:43:16 - 16
22 - 15:44:23 - 17
23 - 15:45:27 - 18
24 - 15:47:48 - 19
25 - 15:48:42 - 20
26 - 15:49:06 - 20
27 - 15:51:05 - 21
28 - 15:52:07 - 21
29 - 15:52:54 - 21
30 - 15:56:30 - 22
31 - 15:58:31 - 22
32 - 16:00:05 - 23
33 - 16:01:02 - 24
34 - 16:02:16 - 25
35 - 16:04:04 - 25
36 - 16:04:53 - 26
37 - 16:06:20 - 27
38 - 16:07:31 - 28
39 - 16:08:55 - 29
40 - 16:11:40 - 30


In [36]:
new_responses[0]

('sokorsak: Intermittenta bröstsmärtor \nkon: M \nalder: 63 \nsocialt: Änkling, två vuxna döttrar, pensionerad brandman. \ntidigare_sjukdomar: Diabetes typ 2, tidigare rökare (slutade för 10 år sedan) \naktuellt: Upplevt obehag och tryck över bröstet i flera veckors tid, mestadels vid ansträngning men senaste dagarna även i vila. Smärtan strålar ut mot vänster arm. Känt sig allmänt svag och tidvis illamående. Ingen synkope. Har tagit extra nitroglycerin de senaste två dagarna, vilket inte gett någon avlastning. Inte haft feber men svettas flitigt, speciellt nattetid. Idag mycket sämre, beslöt att söka akut. \nhereditet: Moder hade angina pectoris, levde till 89 års ålder. \nrokning: Tidigare rökare, slutade för 10 år sedan. \nA: A andning fri, ingen ansträngning vid vila. \nB: AF 18, pox 98% på rumsluft. Ausk pulm ua, ingen ronchi eller wheezing. Normal bröstkorgsrörelse. \nC: Blodtryck 150/95, puls 88. Cap refill ',
 '*Sjukdomshistoria*\nDiabetes typ 2, tidigare rökare. Moder hade ang

In [38]:
df = pd.DataFrame(new_responses,columns=["description","summary"])
v_260 = pd.concat([v_130,df],ignore_index=True)
len(v_260)

211

In [267]:
v_260.to_parquet("synthetic_224_corrected.parquet",compression=None)

In [283]:
i = 2
print(v_260.iloc[i,1]) #Hypertoni behandlad med medicinering

*Patientens namn och personnummer är fingerade för integritetsskäl.*

    <anamnesis>
    sökorsak: Ospecifik buksmärta 
    kön: M 
    ålder: 63 
    socialt: Gift, bor med maka. Pensionerad lärare. 
    tidigare_sjukdomar: Diabetes typ 2. Tidigare gallstenar. Appendektomi som barn. 
    aktuellt: Buksmärtor under flera timmar. Inte kolikartade. Inga tidigare episoder. Avföringen u.a. Inga urologiska symtom. Inget illamående eller kräkningar. 
    alkohol: Måttlig konsumtion vid sociala tillfällen. 
    allmantillstand: Vid besöket lugn och orienterad. 
    buk: Ömhet i höger fossa iliaca utan peritonit. 
    sbp: 135.0 
    dbp: 85.0 
    sat: 98.0 
    puls: 75.0 
    temp: 37.2 
    af: 16.0 
    lokalstatus: Inga anmärkningar. 
    lab: Hb, vita, CRP lätt förhöjd. Normalt glukos. Lever- och pankreasvärden normala. 
    bedömning: Oklar buk. Inlägges för observation. Analgetika vid behov. Återbedömning av kirurg. Iv-vätska. CT-buk planeras. 
    huvuddiagnos_001: R10.4 
    </anam

In [286]:
v_260.iloc[i,1] = """
*Sjukdomshistoria*
63-årig man med diabetes typ 2, tidigare gallstenar och appendektomi som barn.

*Sökorsaker* 
Ospecifik buksmärta under flera timmar utan kolik. Ömhet i höger fossa iliaca.

*Åtgärder*
Inläggning för observation, analgetika vid behov, iv-vätska och planerad CT-buk. Återbedömning av kirurg."""

In [281]:
d = """
sokorsak: Yrsel
kon: K
alder: 65
socialt: Änka. 3 vuxna barn. Pensionerad lärare.
tidigare_sjukdomar: Hyperlipidemi. Hjärtflimmer.
aktuellt: Patienten söker för episoder av yrsel och ostadighet den senaste månaden. Yrseln är rotatorisk och uppkommer i anslutning till snabba huvudrörelser. Har haft ett par falltillbud utan synkope. Känner sig allmänt uppgiven och har minskad aptit. Ingen historik av öronproblem.
hereditet: Far med stroke.
rokning: Aldrig rökt.
allmantillstand: Lätt tröttad men uppträder adekvat.
hjarta: Oregelbunden rytm utan hörbara blåsljud.
lungor: Vesikulärt andningsljud, inga rassel.
buk: Oöm, utan resistenser.
sbp: 145.0
dbp: 85.0
sat: 97.0
puls: 78 oregelbunden.
temp: 36.7
af: 14.0
lokalstatus: Inga anmärkningar.
neurologi: Inga fokala neurologiska bortfall.
lab: CRP 2, LPK 7, TSH 3.5
ekg: Förmaksflimmer utan andra avvikelser.
bedomning: Yrsel troligen av positionell karaktär. Inget akut intrakraniellt. Utredning ger misstanke om benign positionell paroxysmal vertigo (BPPV).
planering: Sjukgymnastik, manöver enl. Epley. Återbesök om 3 veckor.
huvuddiagnos_001: H81.1
"""
s = """
*Sjukdomshistoria*
Hyperlipidemi, hjärtflimmer. Far med stroke. Aldrig rökt.

*Sökorsaker*
Episoder av rotatorisk yrsel och ostadighet, särskilt vid snabba huvudrörelser.

*Åtgärder*
Epleys manöver hos sjukgymnast. Återbesök om 3 veckor.
"""
v_260 = pd.concat([v_260,pd.DataFrame([(d,s)],columns=["description", "summary"])],ignore_index=True)

In [287]:
v_260

,description,summary
0,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,"*Sjukdomshistoria*\nHypertoni, hypotyreos, hyp..."
1,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,<Sjukdomshistoria>\n58-årig kvinnlig patient m...
2,\nsökorsak: Ospecifik buksmärta \n kön: M \...,\n*Sjukdomshistoria*\n63-årig man med diabetes...
3,sokorsak: BrastSmarta \nkon: M \nalder: 51 \ns...,"\n*Sjukdomshistoria*\nHypertoni, Typ 2 diabete..."
4,sokorsak: BukSmarta \nkon: K \nalder: 20 \nsoc...,*Sjukdomshistoria*\nMisstänkt appendicit (K379...
...,...,...
224,\nsokorsak: Buksmärtor\nkon: K\nalder: 45\nsoc...,"\n**Sjukdomshistoria**\n45-årig kvinna, tidiga..."
225,\nsokorsak: Bröstsmärtor\nkon: K\nalder: 68\ns...,\n*Sjukdomshistoria*\nBröstsmärta. Kvinnan är ...
226,\nsokorsak: Återkommande huvudvärk\nkon: M\nal...,\n*Sjukdomshistoria*\nAllergisk astma behandla...
227,\nsokorsak: Yrsel och huvudvärk\nkon: M\nalder...,\n*Sjukdomshistoria*\nIschemisk hjärtsjukdom m...


In [288]:
v_260.to_parquet("synthetic_229_corrected.parquet",compression=None)